In [1]:
from importlib import reload 
import sys
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial.distance import braycurtis
from scipy.special import softmax
import seaborn as sns
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import model_class
reload(model_class)


<module 'model_class' from 'c:\\Users\\tomer\\Desktop\\BSc\\year3\\sem B\\workshop_microbiome\\Milestone2\\model_class.py'>

$$f(D,\Delta_t) = \alpha e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right) D_{t-1} + \beta\left(1- e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right)\right)\cdot \overline{\{D_1,...,D_{t-2}\}}$$

In [2]:
import model_class
reload(model_class)


<module 'model_class' from 'c:\\Users\\tomer\\Desktop\\BSc\\year3\\sem B\\workshop_microbiome\\Milestone2\\model_class.py'>

In [3]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"

In [4]:
metadata_df = pd.read_csv(metadata_path)

In [5]:
metadata_df.columns

Index(['sample', 'baboon_id', 'collection_date', 'sex', 'age', 'social_group',
       'group_size', 'rain_month_mm', 'season', 'hydro_year', 'month',
       'diet_PC1', 'diet_PC2', 'diet_PC3', 'diet_PC4', 'diet_PC5', 'diet_PC6',
       'diet_PC7', 'diet_PC8', 'diet_PC9', 'diet_PC10', 'diet_PC11',
       'diet_PC12', 'diet_PC13'],
      dtype='object')

In [6]:
metadata_df.social_group.unique()

array(['g_1.22', 'g_2.2', 'g_1.1', 'g_1.12', 'g_1.21', 'g_1.211',
       'g_1.11', 'g_2.1', 'g_2.12', 'g_2.11'], dtype=object)

In [7]:
# data_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_data.csv", index_col="sample")
# metadata_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_metadata.csv",  index_col="sample")

In [8]:
super_model = model_class.superModel(data_path, metadata_path)

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\model_class.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean["collection_date"] = (pd.to_datetime(metadata_clean['collection_date']) - pd.Timestamp('1970-01-01')).dt.days


In [9]:
import time
start = time.time()
super_model.fit()
time.time() - start


KeyboardInterrupt



In [9]:
for baboon in super_model.baboons:
    baboon.alpha_ = np.zeros([61,61])
    baboon.beta_ = np.eye(61,61)

In [10]:
baboons = super_model.baboons
super_model.baboons = super_model.baboons[:60]

In [11]:
Baboon_96 = baboons[-1]

In [12]:
last_data = Baboon_96.data[57:]
Baboon_96.data = Baboon_96.data[:57]

In [13]:
res = super_model.predict(Baboon_96.data, Baboon_96.metadata, iterative=True)
res

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:36: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum()


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
[nan

In [17]:
Baboon_78 = baboons[-2]

In [18]:
baboon_78_data = Baboon_78.data[2:]
Baboon_78.data = Baboon_78.data[:2]

In [19]:
baboon_101 = baboons[0]

In [20]:
len(baboon_78_data)

0

In [21]:
# iterative_res = super_model.predict(Baboon_78.data, Baboon_78.metadata, iterative=True)
noninterative_res = super_model.predict(Baboon_78.data, Baboon_78.metadata, iterative=False)

[[1.72715497e+00 0.00000000e+00 9.85018031e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.41630129e+01 0.00000000e+00 1.37804678e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.48104160e+02 0.00000000e+00 5.40715633e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.45551683e+03 0.00000000e+00 8.30099407e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.21809993e+03 0.00000000e+00 6.94697590e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.56465177e+02 0.00000000e+00 2.60327786e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[1.50292176e+00 0.00000000e+00 8.57135032e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.10259754e+01 0.00000000e+00 1.19913761e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.25013618e+02 0.00000000e+00 4.70515561e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.26654987e+03 0.00000000e+00 7.22329196e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning:

[[2.70141549e+00 0.00000000e+00 1.54065096e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.77929824e+01 0.00000000e+00 2.15538095e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.48291457e+03 0.00000000e+00 8.45724681e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.27655061e+03 0.00000000e+00 1.29834522e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.90521064e+03 0.00000000e+00 1.08656540e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.13949889e+02 0.00000000e+00 4.07174530e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[3.53632156e+00 0.00000000e+00 2.01680831e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.94733738e+01 0.00000000e+00 2.82152824e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.94122776e+03 0.00000000e+00 1.10710642e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.98014690e+03 0.00000000e+00 1.69961496e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning:

[[1.17274623e+00 0.00000000e+00 6.68831809e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.64067978e+01 0.00000000e+00 9.35700149e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.43767119e+02 0.00000000e+00 3.67148421e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.88302671e+02 0.00000000e+00 5.63641345e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.27095499e+02 0.00000000e+00 4.71702884e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.09941971e+02 0.00000000e+00 1.76763773e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[1.11699309e+00 0.00000000e+00 6.37035099e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.56268077e+01 0.00000000e+00 8.91216341e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.13162001e+02 0.00000000e+00 3.49693940e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.41318103e+02 0.00000000e+00 5.36845460e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]

...

[[1.56134332e+00 0.00000000e+00 8.90453580e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.18432970e+01 0.00000000e+00 1.24575048e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.57083543e+02 0.00000000e+00 4.88805439e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.31578319e+03 0.00000000e+00 7.50407572e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.10115897e+03 0.00000000e+00 6.28004703e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.12643260e+02 0.00000000e+00 2.35335600e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[1.30863741e+00 0.00000000e+00 7.46332241e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.83079243e+01 0.00000000e+00 1.04412377e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.18363198e+02 0.00000000e+00 4.09691495e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.10282157e+03 0.00000000e+00 6.28952904e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning:

[[1.44972350e+00 0.00000000e+00 8.26795402e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.02817282e+01 0.00000000e+00 1.15669227e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.95810975e+02 0.00000000e+00 4.53860929e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.22171836e+03 0.00000000e+00 6.96761228e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.02243755e+03 0.00000000e+00 5.83108892e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.83143554e+02 0.00000000e+00 2.18511550e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[2.00061219e+00 0.00000000e+00 1.14097409e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.79886976e+01 0.00000000e+00 1.59623035e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.09821572e+03 0.00000000e+00 6.26326127e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.68596608e+03 0.00000000e+00 9.61527493e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)
c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\imports.py:37: RuntimeWarning: invalid value encountered in divide
  return arr/arr.sum(axis=1).reshape(-1,1)


UnboundLocalError: cannot access local variable 'n' where it is not associated with a value

In [37]:
Baboon_78.metadata

,baboon_id,collection_date,sex,age,social_group,group_size,rain_month_mm,season,hydro_year,month,...,diet_PC4,diet_PC5,diet_PC6,diet_PC7,diet_PC8,diet_PC9,diet_PC10,diet_PC11,diet_PC12,diet_PC13
sample,,,,,,,,,,,,,,,,,,,,,
sample_11407-TTACCACATCTA-394,Baboon_78,11659,M,1.117043,g_1.21,22,13.2,wet,2002,12,...,-7.299950,11.585974,-8.259149,-6.951492,13.476441,8.591452,-2.715730,-0.670181,-0.363471,-0.270125
sample_12051-GTGCGCCTTCTG-408,Baboon_78,11725,M,1.297741,g_1.21,21,61.0,wet,2002,2,...,-16.448277,-14.826011,-2.471410,4.679189,0.180357,7.190213,5.145853,-8.074240,0.534689,0.466634
sample_11408-ATTAATGAAGCG-395,Baboon_78,11729,M,1.308693,g_1.21,21,61.0,wet,2002,2,...,-13.131171,-15.033754,-1.162481,3.358549,0.529433,5.814366,3.295151,-11.286747,0.469203,0.410511
sample_12050-CAGTAGCGATAT-407,Baboon_78,11740,M,1.338809,g_1.21,21,2.0,wet,2002,2,...,-10.538072,-18.027935,-4.024002,-3.760225,1.617033,2.790060,-0.988068,-8.916010,0.414878,0.198270
sample_11407-TCAATACAATAT-394,Baboon_78,11834,M,1.596167,g_1.21,24,51.0,wet,2002,5,...,7.244653,-5.417804,-2.619896,-3.011515,-2.509070,4.895551,-2.253855,-2.144438,0.211195,0.092890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_11407-AGGGACTTCAAT-394,Baboon_78,15045,M,10.387410,g_2.2,113,31.0,wet,2011,3,...,-0.887084,-0.887129,1.793682,2.890806,0.489460,3.894504,1.639758,-0.830671,0.445997,-0.001310
sample_11410-GTAGCACTCATG-396,Baboon_78,15118,M,10.587270,g_2.2,114,3.2,wet,2011,5,...,-1.796998,1.102359,-0.692042,0.505585,1.852034,0.843750,-0.483853,-0.338856,0.562867,0.113109
sample_12049-GTTAACTTACTA-407,Baboon_78,15188,M,10.778920,g_2.2,116,0.0,dry,2011,8,...,-3.741875,3.188046,-3.503696,2.028410,-1.355503,-1.415436,-0.842528,-0.896124,0.455143,0.085074


In [14]:
len(iterative_res)

NameError: name 'iterative_res' is not defined

In [15]:
braycurtis_78_iterative = 0
for i in range(111):
    braycurtis_78_iterative += braycurtis(iterative_res[i], baboon_78_data.iloc[i].values)

braycurtis_78_non_iterative = 0
for i in range(111):
    braycurtis_78_non_iterative += braycurtis(noninterative_res[i], baboon_78_data.iloc[i].values)

NameError: name 'iterative_res' is not defined

In [16]:
x = 0
for i in range(10):
    x += braycurtis(res[i], last_data.iloc[i].values)

NameError: name 'res' is not defined

In [17]:
x

0

In [18]:
res

NameError: name 'res' is not defined

In [30]:
non_iter_res = super_model.predict(Baboon_96.data, Baboon_96.metadata, iterative=False)

In [25]:
y = 0
for i in range(10):
    y += braycurtis(non_iter_res[i], last_data.iloc[i].values)

In [26]:
y

3.0114451737451184

In [11]:
baboon_101 = super_model.baboons[0]

In [12]:
baboon_101.fit(0)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[ 9.99560751e-01, -7.52734711e-05, -6.95880161e-05, ...,
         -2.14871154e-05, -1.26788573e-04, -5.54778395e-05],
        [-2.42678641e-04,  9.99945119e-01, -1.70861341e-05, ...,
         -1.49847119e-05, -6.01530891e-05, -4.34695731e-05],
        [-7.14703165e-05, -7.50187236e-05,  9.99941819e-01, ...,
         -1.52833813e-05, -1.21462790e-04, -7.36542145e-05],
        ...,
        [-2.84972081e-04, -5.47160373e-05, -1.75448747e-05, ...,
          9.99983832e-01, -6.24160466e-05, -4.34661569e-05],
        [-2.67673170e-04, -5.78305930e-05, -4.19201036e-05, ...,
         -1.54009945e-05,  9.99908403e-01, -2.34738525e-05],
        [-1.73669419e-04, -4.10358079e-05, -2.36890702e-05, ...,
         -1.13089311e-05, -6.84743502e-05, 

In [13]:
# calculate the objective function

'''for the ith row in self.data (from row 3)
1. calculate mean of previous i-2 rows = d_mean
2. calculate time difference between ith row and i-1th row = d_time
calculate the prediction for the ith row using the formula'''


'''
calculate difference between prediction and actual value using bray-curtis dissimilarity and return the sum/mean -TBD'''
self = baboon_101
lambda_ = 0
method="counts"
alpha = baboon_101.alpha_
beta = baboon_101.beta_


D_t1 = self.transformed_data[1:-1].values
D_mean = self.df_cumulative_mean[:-2].values
cos = 0# np.cos((2*np.pi*self.delta_t[2:].values)/365)
exp = np.exp(-lambda_*self.delta_t[2:].values)

f = alpha@(exp*cos*D_t1.T) + beta@((1-exp*cos)*D_mean.T)

f = to_composition(f.T, type = method) # transpose f to match the shape of D - each row is a sample

# calculate bray-curtis dissimilarity
bc =  np.array([braycurtis(self.data.values[i+2], f[i]) for i in range(len(f))])

f.shape

(51, 61)